In [12]:
import pyodbc
import pandas as pd
# insert data from csv file into dataframe.
# working directory for csv file: type "pwd" in Azure Data Studio or Linux


# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '172.17.0.2' 
driver = [item for item in pyodbc.drivers()][-1]
database = 'AnimeWebApp' 
username = 'sa' 
password = 'Thatkhiem123' 
cnxn = pyodbc.connect('DRIVER=' + driver +';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password +';TrustServerCertificate=yes')
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:


In [15]:
producers = pd.read_csv("../../data/csv/output/producers.csv")

In [30]:
cursor.execute('''
		CREATE TABLE Producer (
			producer_id int primary key,
			producer_name nvarchar(100),
			)
               ''')
cursor.commit()

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]There is already an object named 'Producer' in the database. (2714) (SQLExecDirectW)")

In [14]:
for index, row in producers.iterrows():
     cursor.execute("INSERT INTO Producer VALUES(?,?)", row.producer_id, row.producer_name)
cnxn.commit()


In [63]:
anime = pd.read_csv("../../data/csv/output/anime.csv")
anime.columns

Index(['anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'duration', 'rating', 'score', 'scored_by', 'rank', 'popularity',
       'members', 'favorites', 'premiered', 'licensor', 'studio', 'aired_from',
       'aired_to'],
      dtype='object')

In [66]:
import numpy as np
anime = anime.fillna(np.nan).replace([np.nan], [None])

# Reference https://stackoverflow.com/questions/66579476/pyodbc-csv-to-sql-table-how-to-handle-nan-values

In [45]:
cursor.execute('''DROP TABLE Anime''')
cursor.commit()

In [46]:
cursor.execute(
    '''
    CREATE TABLE Anime (
        anime_id int primary key,
        title nvarchar(1000),
        title_english nvarchar(1000),
        title_japanese nvarchar(1000), 
        title_synonyms nvarchar(1000),
        image_url nvarchar(500),
        type nvarchar(50),
        source nvarchar(50),
        episodes int,
        status nvarchar(50),
        duration nvarchar(100),
        rating nvarchar(100),
        score float,
        scored_by int,
        rank float, 
        popularity int, 
        members int,
        favorites int,
        premiered nvarchar(50),
        licensor nvarchar(100),
        studio nvarchar(100),
        aired_from date,
        aired_to date
    )
    '''
)
cursor.commit()

In [65]:
for index, row in anime.iterrows():
      cursor.execute("INSERT INTO Anime VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.anime_id, row.title, row.title_english, row.title_japanese,
       row.title_synonyms, row.image_url, row.type, row.source, row.episodes, row.status,
       row.duration, row.rating, row.score, row.scored_by, row["rank"], row.popularity,
       row.members, row.favorites, row.premiered, row.licensor, row.studio, row.aired_from,
       row.aired_to)

       
cnxn.commit()

In [67]:
genres = pd.read_csv("../../data/csv/output/genre.csv")


In [68]:
cursor.execute('''
		CREATE TABLE Genre (
			genre_id int primary key,
			genre_name nvarchar(100),
			)
               ''')
cursor.commit()

In [69]:
for index, row in genres.iterrows():
     cursor.execute("INSERT INTO Genre VALUES(?,?)", row.genre_id, row.genre_name)
cnxn.commit()

In [70]:
music = pd.read_csv("../../data/csv/output/music.csv")

In [76]:
cursor.execute('''
    DROP TABLE Music
''')

In [77]:
cursor.execute('''
		CREATE TABLE Music (
			anime_id int references Anime(anime_id),
			song_name nvarchar(1000),
            type nvarchar(50) check (type in ('opening_theme', 'ending_theme'))
			)
               ''')
cursor.commit()

In [78]:
for index, row in music.iterrows():
     cursor.execute("INSERT INTO Music VALUES(?,?,?)", row.anime_id, row.song_name, row.type)
cnxn.commit()

In [79]:
produces = pd.read_csv("../../data/csv/output/produces.csv")

In [89]:
cursor.execute('''
		DROP TABLE Produces
               ''')
cursor.commit()

In [91]:
cursor.execute('''
		CREATE TABLE Produces (
			anime_id int references Anime(anime_id),
			producer_id int references Producer(producer_id),
			)
               ''')
cursor.commit()

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]There is already an object named 'Produces' in the database. (2714) (SQLExecDirectW)")

In [92]:
for index, row in produces.iterrows():
     try:
          cursor.execute("INSERT INTO Produces VALUES(?,?)", str(row.anime_id), str(row.producer_id))
     except:
          continue
cnxn.commit()

In [93]:
related = pd.read_csv("../../data/csv/output/related.csv")

In [94]:
cursor.execute('''
		CREATE TABLE Related (
			anime_id int references Anime(anime_id),
			related_anime_id int references Anime(anime_id),
            relation_type nvarchar(50)
			)
               ''')
cursor.commit()

In [96]:
for index, row in related.iterrows():
     try:
          cursor.execute("INSERT INTO Related VALUES(?,?,?)", str(row.anime_id), str(row.related_anime_id), str(row.relation_type))
     except:
          continue
cnxn.commit()

In [97]:
classifies = pd.read_csv("../../data/csv/output/classifies.csv")

In [98]:
cursor.execute('''
		CREATE TABLE Classifies (
			anime_id int references Anime(anime_id),
			genre_id int references Genre(genre_id),
			)
               ''')
cursor.commit()

In [99]:
for index, row in classifies.iterrows():
     try:
          cursor.execute("INSERT INTO Classifies VALUES(?,?)", str(row.anime_id), str(row.genre_id))
     except:
          continue
cnxn.commit()

In [108]:
cursor.execute('''
    CREATE TABLE Users (
        user_id int identity(1,1) primary key,
        username nvarchar(50) not null,
        email nvarchar(100) not null,
        password nvarchar(100),
        role nvarchar(50) check (role in ('admin', 'mod', 'user'))
    )
''')

cursor.commit()

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]There is already an object named 'Users' in the database. (2714) (SQLExecDirectW)")

In [110]:
cursor.execute('''
    DROP TABLE Rating
''')

In [111]:
cursor.execute('''
    CREATE TABLE Rating (
        user_id int references Users(user_id),
        anime_id int references Anime(anime_id),
        rating_star int check (rating_star between 0 AND 10), 
        CONSTRAINT PK_Rating PRIMARY KEY (user_id, anime_id)
    )
    ''')

In [112]:
cursor.execute(
    '''
    CREATE TABLE WatchList (
        user_id int references Users(user_id),
        anime_id int references Anime(anime_id),
        episode int
    )
    '''
)

In [113]:
cursor.execute(
    '''
    CREATE TABLE Review (
        user_id int references Users(user_id),
        anime_id int references Anime(anime_id),
        content nvarchar(max)
    )
    '''
)

In [115]:
cursor.execute(
    '''
    CREATE TABLE Favorite (
        user_id int references Users(user_id),
        anime_id int references Anime(anime_id)

    )
    '''
)